In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi)

In [5]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

# 有效像元

In [7]:
# 有效像元的计算（方案一）
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
vis = {'bands': ['count'], 'palette': ['ffffff', 'dd09ec'], 'min': 0.0, 'max': 35.0, 'opacity': 1.0}
Map.addLayer(validPixel_layer,vis,'validPixel')

# 有效像元的计算（方案二）
valid_layer = landsat_images.map(lambda i : i.clip(roi).select('B3').gte(0)).sum()
vis = {'bands': ['B3'], 'palette': ['ffffff', 'blue'], 'min': 0.0, 'max': 35.0, 'opacity': 1.0}
Map.addLayer(valid_layer,vis,'valid layer')

# 水体频率

In [8]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")

# 计算水体频率
## 计算水体识别为水体的个数
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
## 计算有效像元的个数
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
vis = {'bands': ['count'], 'palette': ['ffffff', 'dd09ec'], 'min': 0.0, 'max': 35.0, 'opacity': 1.0}
# Map.addLayer(validPixel_layer,vis,'validPixel')
## 水体频率
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
# Map.addLayer(index_waterfrequency,{'palette':['white','blue'],'min':0,'max':1},'index water frequency')

# 永久水体；frequency >= 0.75
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
# Map.addLayer(index_permanent,{'palette':['white','green'],'min':0,'max':2},"index permanent")

# 季节性水体；25 =< frequency < 75
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
# Map.addLayer(index_season,{'palette':['white','cyan'],'min':0,'max':1},'index season')
# Map.addLayer(index_season,{'palette':['red']},'index season')

# 水体分类；“0：陆地；1：季节性水体；2：永久性水体”
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
Map.addLayer(index_waterclass,{'palette':['white','red','blue'],'min':0,'max':2},"index waterclass")